In [2]:
from selenium.webdriver import Chrome
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pymongo
from selenium.common.exceptions import NoSuchElementException
from cost_of_living import *
from functools import reduce
import matplotlib.pyplot as plt
%matplotlib inline
from hidden import debt

In [10]:
data = pd.read_excel('data/citydf.xlsx')

### Check Data for missing values --> Which appear to be denoted as '?'

In [20]:
data[data['price'] == '?']

,category,city,item,price
28,Transportation,Palo-Alto,Monthly Pass (Regular Price),?
796,Markets,Mountain-View,Cigarettes 20 Pack (Marlboro),?
1240,Transportation,Irvine,Taxi 1 mile (Normal Tariff),?
1515,Transportation,Plano,Taxi 1 mile (Normal Tariff),?
1516,Transportation,Plano,Taxi 1hour Waiting (Normal Tariff),?


In [ ]:
### Write function to find lat_long of a city

In [32]:
from geopy.geocoders import Nominatim
def find_lat_long(city):
    geolocator = Nominatim(user_agent="LifestyleDesign")
    location = geolocator.geocode(city)
    return location[1]

In [ ]:
data['lat_long'] = data['city'].apply(find_lat_long)

In [54]:
cities = data.city.unique()

In [57]:
city_lat_long = {}
for city in cities:
    print(city)
    print(find_lat_long(city))
    city_lat_long[city] = find_lat_long(city)

Palo-Alto
(37.4455862, -122.1619289)
San-Jose
(37.3361905, -121.8905833)
Portland
(45.5202471, -122.6741949)
Austin
(30.30758105, -97.712630246401)
Denver
(39.7391428, -104.984696)
San-Diego
(32.7174209, -117.1627714)
Washington D.C.
(38.8950092, -77.0365625)
Boston
(42.3602534, -71.0582912)
San-Francisco
(37.7647993, -122.4629897)
Seattle
(47.6038321, -122.3300624)
Atlanta
(33.7490987, -84.3901849)
Los-Angeles
(34.0536834, -118.2427669)
Chicago
(41.8755616, -87.6244212)
New-York
(40.7308619, -73.9871558)
Mountain-View
(37.3893889, -122.0832101)
Santa-Monica
(34.019657, -118.487549)
Cambridge
(52.2034823, 0.1235817)
Salt-Lake-City
(40.7670126, -111.8904308)
Raleigh
(35.7803977, -78.6390989)
Nashville
(36.1622296, -86.7743531)
Sunnyvale
(37.3688301, -122.0363496)
Boulder
(40.0149856, -105.2705456)
Irvine
(55.6143018, -4.6655544)
Philadelphia
(39.9524152, -75.1635755)
Dallas
(32.7762719, -96.7968559)
Bellevue
(47.6144219, -122.1923372)
Charlotte
(35.2270869, -80.8431268)
Plano
(33.013676

In [52]:
data['city'].replace('Washington', "Washington D.C.", inplace=True)

,category,city,item,price
0,Restaurants,Palo-Alto,"Meal, Inexpensive Restaurant",15.45
1,Restaurants,Palo-Alto,"Meal for 2 People, Mid-range Restaurant, Three...",72.50
2,Restaurants,Palo-Alto,McMeal at McDonalds (or Equivalent Combo Meal),8.00
3,Restaurants,Palo-Alto,Domestic Beer (1 pint draught),6.00
4,Restaurants,Palo-Alto,Imported Beer (11.2 oz small bottle),7.00
5,Restaurants,Palo-Alto,Cappuccino (regular),3.80
6,Restaurants,Palo-Alto,Coke/Pepsi (11.2 oz small bottle),1.95
7,Restaurants,Palo-Alto,Water (11.2 oz small bottle),1.68
8,Markets,Palo-Alto,"Milk (regular), (1 gallon)",4.33
9,Markets,Palo-Alto,Loaf of Fresh White Bread (1 lb),3.33
